# 第2章 確率・統計の基礎

In [ ]:
import os
DATA_DIR = "LNPR_BOOK_CODES/sensor_data/"
os.chdir(DATA_DIR)

## 2.1 センサデータの収集とJupyter Notebook上での準備

In [ ]:
import pandas as pd
data  = pd.read_csv("sensor_data_200.txt", delimiter=" ", 
                    header=None, names = ("date","time","ir","lidar"))
data

In [ ]:
print(data["lidar"][0:5])

## 2.2 度数分布と確率分布

### 2.2.1 ヒストグラムの描画

In [ ]:
import matplotlib.pyplot as plt
data["lidar"].hist(bins=max(data["lidar"])-min(data["lidar"]), align="left")
plt.show()